# 19. lab 43 - obliczenia


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

# ==========================================
#              DANE POMIAROWE
# ==========================================

# --- 1. Drut oporowy (Resistor) ---
# U: 4 miejsca po przecinku (0.0001 V)
# I: Tabela ma 2 miejsca po przecinku (0.01 A) -> niepewność 0.01 A
U_R = np.array([0.0001, 0.1092, 0.1985, 0.2902, 0.4029, 0.5119, 0.6001, 0.6920, 
                0.8000, 0.9000, 1.0090, 1.1000, 1.2000, 1.3000, 1.3990, 1.5000])
I_R_mA = np.array([0.01, 0.02, 0.03, 0.04, 0.06, 0.07, 0.08, 0.09, 
                   0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17]) * 1000

u_U_R = 0.0001           # Niepewność napięcia
u_I_R_mA = 0.01 * 1000   # Niepewność prądu (10 mA) - wynikająca z zapisu w tabeli

# --- 2. Dioda (Diode) ---
# U: 4 miejsca po przecinku
# I: 4 miejsca po przecinku -> niepewność 0.0001 A
U_D = np.array([0.5699, 0.6744, 0.7296, 0.8083, 0.8788, 0.8797, 0.8199, 0.8406, 
                0.8600, 0.8705, 0.8973, 0.9139, 0.9314, 0.9501, 0.9699, 0.9904, 
                0.9150, 0.9270, 0.9370, 0.9490, 0.9610, 0.9720])
I_D_mA = np.array([0.0001, 0.0050, 0.0101, 0.0200, 0.0300, 0.0400, 0.0501, 0.0600, 
                   0.0702, 0.0800, 0.0900, 0.1002, 0.1100, 0.1200, 0.1299, 0.1398, 
                   0.1503, 0.1603, 0.1699, 0.1803, 0.1902, 0.1999]) * 1000

u_U_D = 0.0001
u_I_D_mA = 0.0001 * 1000 # 0.1 mA

# --- 3. Żarówka (Bulb) ---
# U: 4 miejsca po przecinku
# I: 4 miejsca po przecinku -> niepewność 0.0001 A
U_B = np.array([0.0150, 0.0345, 0.0713, 0.1125, 0.1580, 0.2149, 0.3020, 0.4179, 
                0.5811, 0.7472, 0.9242, 0.9960, 1.1670, 1.3690, 1.5740, 1.7830, 
                2.0100, 2.2350, 2.4880, 2.7430, 3.0030])
I_B_mA = np.array([0.0002, 0.0101, 0.0200, 0.0303, 0.0401, 0.0500, 0.0600, 0.0701, 
                   0.0803, 0.0900, 0.1001, 0.1104, 0.1198, 0.1303, 0.1404, 0.1501, 
                   0.1602, 0.1698, 0.1801, 0.1901, 0.2000]) * 1000

u_U_B = 0.0001
u_I_B_mA = 0.0001 * 1000 # 0.1 mA

# ==========================================
#              GENEROWANIE WYKRESÓW
# ==========================================

output_dir = "19-lab-43/report/img"
os.makedirs(output_dir, exist_ok=True)
results = {}

def format_latex(val, unc):
    if unc == 0: return f"{val:.4f}", "0"
    magnitude = np.floor(np.log10(unc))
    digits_to_round = -int(magnitude) + 1
    u_rounded = round(unc, digits_to_round)
    v_rounded = round(val, digits_to_round)
    fmt = f"{{:.{max(0, digits_to_round)}f}}"
    return fmt.format(v_rounded).replace('.', ','), fmt.format(u_rounded).replace('.', ',')

# --- WYKRES 1: OPORNIK (Resistor) ---
if len(U_R) > 2:
    I_R = I_R_mA * 1e-3
    res = stats.linregress(U_R, I_R)
    G, b, std_err = res.slope, res.intercept, res.stderr
    R_calc = 1.0 / G
    u_R = (R_calc**2) * std_err
    results['R'] = R_calc
    results['u_R'] = u_R
    results['G'] = G
    results['b'] = b

    plt.figure(figsize=(10, 6))
    # Errorbar z "krzyżami" (xerr i yerr)
    plt.errorbar(U_R, I_R_mA, xerr=u_U_R, yerr=u_I_R_mA, 
                 fmt='ko', ecolor='black', elinewidth=1, capsize=3, label='Dane pomiarowe')
    
    # Linia dopasowania
    U_lin = np.linspace(0, max(U_R)*1.05, 100)
    I_lin_mA = (G * U_lin + b) * 1e3
    plt.plot(U_lin, I_lin_mA, 'r-', linewidth=1.5, label='Dopasowanie liniowe')
    
    plt.xlabel(r'$U$ [\unit{\volt}]')
    plt.ylabel(r'$I$ [\unit{\milli\ampere}]')
    plt.title('Charakterystyka I(U) drutu oporowego')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.savefig(f"{output_dir}/plot_resistor.png", dpi=300, bbox_inches='tight')
    plt.close()

# --- WYKRES 2: ŻARÓWKA (Bulb) ---
if len(U_B) > 0:
    I_B = I_B_mA * 1e-3
    idx1, idx2 = 8, 9 # Punkty dla ~0.6V
    Rd = (U_B[idx2] - U_B[idx1]) / (I_B[idx2] - I_B[idx1])
    
    # Propagacja niepewności
    u_dU = np.sqrt(2 * u_U_B**2)
    u_dI = np.sqrt(2 * (u_I_B_mA/1000)**2)
    dU, dI = (U_B[idx2] - U_B[idx1]), (I_B[idx2] - I_B[idx1])
    u_Rd = Rd * np.sqrt((u_dU/dU)**2 + (u_dI/dI)**2)
    
    results['Rd'] = Rd
    results['u_Rd'] = u_Rd

    plt.figure(figsize=(10, 6))
    # Errorbar z "krzyżami"
    plt.errorbar(U_B, I_B_mA, xerr=u_U_B, yerr=u_I_B_mA, 
                 fmt='bo', ecolor='black', elinewidth=1, capsize=3, label='Dane pomiarowe')
    
    # Zaznaczenie punktów Rd
    plt.plot(U_B[idx1:idx2+1], I_B_mA[idx1:idx2+1], 'r.', markersize=12, label='Zakres dla $R_d$')
    
    plt.xlabel(r'$U$ [\unit{\volt}]')
    plt.ylabel(r'$I$ [\unit{\milli\ampere}]')
    plt.title('Charakterystyka I(U) żarówki')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.savefig(f"{output_dir}/plot_bulb.png", dpi=300, bbox_inches='tight')
    plt.close()

# --- WYKRES 3: DIODA (Diode) ---
if len(U_D) > 0:
    plt.figure(figsize=(10, 6))
    # Errorbar z "krzyżami"
    plt.errorbar(U_D, I_D_mA, xerr=u_U_D, yerr=u_I_D_mA, 
                 fmt='go', ecolor='black', elinewidth=1, capsize=3, label='Dane pomiarowe')
    
    plt.xlabel(r'$U$ [\unit{\volt}]')
    plt.ylabel(r'$I$ [\unit{\milli\ampere}]')
    plt.title('Charakterystyka I(U) diody')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.savefig(f"{output_dir}/plot_diode.png", dpi=300, bbox_inches='tight')
    plt.close()

# Wypisanie wyników do wklejenia w raporcie (dla pewności)
print("=== RESULTS ===")
if 'R' in results:
    r, ur = format_latex(results['R'], results['u_R'])
    g, _ = format_latex(results['G'], 0)
    b, _ = format_latex(results['b'], 0)
    print(f"R: {r}, u(R): {ur}")
    print(f"Eq: I = {g} U + {b}")

if 'Rd' in results:
    rd, urd = format_latex(results['Rd'], results['u_Rd'])
    print(f"Rd: {rd}, u(Rd): {urd}")
print("=== END ===")